# Create a User Defined Model using astropy.modeling

## Authors
Rocio Kiman, Lia Corrales and Zé Vinícius.

## Learning Goals
* Know and understand tools to make user defined fitters with `astropy` and in which cases it could be useful
* Learn custom fitters
* This tutorial assumes the student knows how to fit data using `astropy.modeling`. Check the first tutorial in case you are not familiar with it [here](https://github.com/astropy/astropy-tutorials/tree/master/tutorials/notebooks/Models-Quick-Fit).

## Keywords
Modeling, User Defined Fitter, Custom Fitter 

## Summary
In this tutorial, we will learn how to define a new fitter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling.fitting import (_validate_model,
                                      _fitter_to_model_params,
                                      _model_to_fit_params, Fitter,
                                      _convert_input)
from astropy.modeling.optimizers import Simplex
from astropy.modeling import models, fitting
from scipy import optimize

from astropy.modeling.optimizers import (SLSQP, Simplex)
from astropy.modeling.statistic import (leastsquare)

class SLSQPFitter(Fitter):
    def __init__(self):
        super().__init__(optimizer=SLSQP, statistic=leastsquare)
                        
    def errorfunc(self, fps, *args):
        meas = args[0]
        self.fitpars = fps
        res = model(*args[1:]) - meas
        return np.sum(res**2)
    
    def __call__(self, model, x, y , maxiter=100, epsilon=10**(-12)):
        b = [model.bounds[key] for key in model.param_names]
        self.fitpars = optimize.fmin_slsqp(self.errorfunc, x0=model.parameters[:],
        args=(y, x), bounds=b)
        return model

x = np.linspace(0,1,100)
y = 2*x
y_err = np.random.normal(0,0.05,100)
x_err = np.random.normal(0,0.05,100)

model = models.Linear1D()
fitter = SLSQPFitter() 
best_fit2 = fitter(model, x, y)

plt.errorbar(x,y,yerr=y_err,xerr=x_err,fmt='.',zorder=0)
plt.plot(x,best_fit2(x),'r',zorder=1)
plt.show()